In [1]:
import json
import copy
import yoll.chessboard
from yoll.chessboard import *

dicts 初始化成功


导入棋谱，查看相关信息

In [2]:
# 读取棋谱文件
cm_file = open('../data/raw_chess_manual.json', 'r')
# 读取文件所有内容
json_str = cm_file.read()

# 转化成 json_dict
cm_json = json.loads(json_str)

half_game = 0
full_game = 0

for i in range(0, len(cm_json)):
    if cm_json[i]['init'] == '':
        full_game += 1
    else:
        half_game += 1

print('half game nums: ', half_game)
print('full game nums: ', full_game)

half game nums:  2861
full game nums:  6598


筛选出全局棋谱，棋谱路径坐标进行变换

In [3]:
fout = open('../data/full_game.txt', 'w')
row_num = 0

for i in range(0, len(cm_json)):
    if cm_json[i]['init'] == '':
        move_list = cm_json[i]['move_list']
        move_list_f = ''
        for j in range(0, len(move_list), 2):
            move_list_f += move_list[j + 1]
            move_list_f += move_list[j]
        fout.write(move_list_f)
        fout.write('\n')
        row_num += 1

print(row_num)

6598


In [4]:

def gen_dataset(move_list: str):
    out_self = open('../data/self_move.txt', 'a')
    out_rival_temp = open('../data/rival_move_temp.txt', 'a')
    board = Chessboard()

    for i in range(0, len(move_list), 8):

        out_self.write(board.to_board_sequence() + '#')  # 写棋盘

        fx, fy, tx, ty = int(move_list[i]), int(move_list[i + 1]), int(move_list[i + 2]), int(move_list[i + 3])
        path = ChessPath(fx, fy, tx, ty)
        out_self.write(str(path) + '\n')  # 写路径

        board.move_chess(path)

        if i + 4 < len(move_list):
            out_rival_temp.write(board.to_board_sequence() + '#')
            fx, fy, tx, ty = int(move_list[i + 4]), int(move_list[i + 5]), int(move_list[i + 6]), int(move_list[i + 7])
            path = ChessPath(fx, fy, tx, ty)

            out_rival_temp.write(str(path) + '\n')
            board.move_chess(path)

    out_self.close()
    out_rival_temp.close()


In [5]:
# 注释掉以免后面重复生成
# in_full_game = open('../data/full_game.txt', 'r')
#
# all_lines = in_full_game.readlines()
#
# # for i in [all_lines[90], all_lines[91]]:
# for i in all_lines:
#     path = i.strip('\n')
#     gen_dataset(path)

将棋谱进行变换，将对手棋谱调整为第一视角，将双方棋谱进行对称增强

In [5]:
# 生成 self_move_sym
import time
from yoll.chessboard import *

fout = open('../data/self_move_sym.txt', 'a')
fin = open('../data/self_move.txt', 'r')

all_lines = fin.readlines()

for line in all_lines:
    board = Chessboard(board_seq=line.split('#')[0])
    path = ChessPath(path_str=line.split('#')[1])
    # board.print_chessboard()
    # print('path: ', line.split('#')[1])
    board.symmetry()
    path.symmetry()
    fout.write(board.to_board_sequence() + '#')
    fout.write(str(path) + '\n')

生成 rival_move

In [6]:
# 生成 rival_move
fin = open('../data/rival_move_temp.txt', 'r')
fout = open('../data/rival_move.txt', 'a')

all_lines = fin.readlines()

for line in all_lines:
    board = Chessboard(board_seq=line.split('#')[0])
    path = ChessPath(path_str=line.split('#')[1])

    board.side_exchange()
    path.side_exchange()

    fout.write(board.to_board_sequence() + '#')
    fout.write(str(path) + '\n')


生成 rival_move_sym

In [7]:
# 生成 rival_move_sym
fout = open('../data/rival_move_sym.txt', 'a')
fin = open('../data/rival_move.txt', 'r')

all_lines = fin.readlines()

for line in all_lines:
    board = Chessboard(board_seq=line.split('#')[0])
    path = ChessPath(path_str=line.split('#')[1])
    # board.print_chessboard()
    # print('path: ', line.split('#')[1])
    board.symmetry()
    path.symmetry()
    fout.write(board.to_board_sequence() + '#')
    fout.write(str(path) + '\n')